In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Nhập dữ liệu 

In [3]:
# data_train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
# data_test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

data_train = pd.read_csv("train.csv")
data_test = pd.read_csv("test.csv")

In [4]:
print("dữ liệu huấn luyện: \n");
data_train.info


dữ liệu huấn luyện: 



<bound method DataFrame.info of         Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0        1          60       RL         65.0     8450   Pave   NaN      Reg   
1        2          20       RL         80.0     9600   Pave   NaN      Reg   
2        3          60       RL         68.0    11250   Pave   NaN      IR1   
3        4          70       RL         60.0     9550   Pave   NaN      IR1   
4        5          60       RL         84.0    14260   Pave   NaN      IR1   
...    ...         ...      ...          ...      ...    ...   ...      ...   
1455  1456          60       RL         62.0     7917   Pave   NaN      Reg   
1456  1457          20       RL         85.0    13175   Pave   NaN      Reg   
1457  1458          70       RL         66.0     9042   Pave   NaN      Reg   
1458  1459          20       RL         68.0     9717   Pave   NaN      Reg   
1459  1460          20       RL         75.0     9937   Pave   NaN      Reg   

     LandContour Ut

In [5]:
data_train.head(2)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500


In [6]:

print("\n dữ liệu test: \n");
data_test.info


 dữ liệu test: 



<bound method DataFrame.info of         Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0     1461          20       RH         80.0    11622   Pave   NaN      Reg   
1     1462          20       RL         81.0    14267   Pave   NaN      IR1   
2     1463          60       RL         74.0    13830   Pave   NaN      IR1   
3     1464          60       RL         78.0     9978   Pave   NaN      IR1   
4     1465         120       RL         43.0     5005   Pave   NaN      IR1   
...    ...         ...      ...          ...      ...    ...   ...      ...   
1454  2915         160       RM         21.0     1936   Pave   NaN      Reg   
1455  2916         160       RM         21.0     1894   Pave   NaN      Reg   
1456  2917          20       RL        160.0    20000   Pave   NaN      Reg   
1457  2918          85       RL         62.0    10441   Pave   NaN      Reg   
1458  2919          60       RL         74.0     9627   Pave   NaN      Reg   

     LandContour Ut

In [7]:
data_test.head(2)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal


Tiền xử lý dữ liệu

In [8]:
def pourcentage(data):
    n = data.shape[0]
    ret = pd.DataFrame(data.isnull().sum(), columns=['missing_number'])
    ret['types'] = data.dtypes # xác đinh kiểu dữ liệu
    ret['duplicate'] = data.duplicated(keep=False).sum() # xác định các hàng trùng lặp
    ret['NAN'] = data.isna().sum() # tổng số các dòng NaN
    return ret

In [9]:
pourcentage(data_train).T

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
missing_number,0,0,0,259,0,0,1369,0,0,0,...,0,1453,1179,1406,0,0,0,0,0,0
types,int64,int64,object,float64,int64,object,object,object,object,object,...,int64,object,object,object,int64,int64,int64,object,object,int64
duplicate,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NAN,0,0,0,259,0,0,1369,0,0,0,...,0,1453,1179,1406,0,0,0,0,0,0


In [10]:
pourcentage(data_test)

,missing_number,types,duplicate,NAN
Id,0,int64,0,0
MSSubClass,0,int64,0,0
MSZoning,4,object,0,4
LotFrontage,227,float64,0,227
LotArea,0,int64,0,0
...,...,...,...,...
MiscVal,0,int64,0,0
MoSold,0,int64,0,0
YrSold,0,int64,0,0
SaleType,1,object,0,1


In [11]:
def compare(train, test):
    dataset_train_nans = train.isnull().sum().to_frame().T # tổng số lượng có đối tượng null trong mỗi hàng 
    dataset_train_nans['DataFrame'] = 'data_train'
    
    dataset_test_nans = test.isnull().sum().to_frame().T
    dataset_test_nans['DataFrame'] = 'data_test'

    combined_data = pd.concat([dataset_train_nans, dataset_test_nans], axis=0, ignore_index=True)
    return combined_data

In [12]:
compare(data_train,data_test)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,DataFrame
0,0,0,0,259,0,0,1369,0,0,0,...,1453,1179,1406,0,0,0,0,0,0.0,data_train
1,0,0,4,227,0,0,1352,0,0,2,...,1456,1169,1408,0,0,0,1,0,NaN,data_test


In [13]:
def xac_dinh_cac_cot_kieu_ky_tu(x):
    categorical_columns_train = x.select_dtypes(include=["category","object"]).columns.tolist();
    return categorical_columns_train
def xac_dinh_cac_cot_kieu_so(x):
    numberic_columns_train = x.select_dtypes(include=["float64","int64"]).columns.tolist();
    return numberic_columns_train
   

In [14]:
print(xac_dinh_cac_cot_kieu_ky_tu(data_train))
cac_cot_kieu_ky_tu_train = xac_dinh_cac_cot_kieu_ky_tu(data_train)
cac_cot_kieu_so_train = xac_dinh_cac_cot_kieu_so(data_train)

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


In [15]:
phan_tram_thieu = data_train.isnull().sum() * 100 / len(data_train)
phan_tram_thieu = phan_tram_thieu[phan_tram_thieu > 0].sort_values(ascending=False)
print(phan_tram_thieu)

PoolQC          99.520548
MiscFeature     96.301370
Alley           93.767123
Fence           80.753425
MasVnrType      59.726027
FireplaceQu     47.260274
LotFrontage     17.739726
GarageType       5.547945
GarageYrBlt      5.547945
GarageFinish     5.547945
GarageQual       5.547945
GarageCond       5.547945
BsmtFinType2     2.602740
BsmtExposure     2.602740
BsmtFinType1     2.534247
BsmtCond         2.534247
BsmtQual         2.534247
MasVnrArea       0.547945
Electrical       0.068493
dtype: float64


In [16]:
loai_cot = phan_tram_thieu[phan_tram_thieu > 0].index.tolist()
phan_tram_thieu.drop(columns=loai_cot, inplace=True) #This line excludes features_to_exclude list of columns from training data
loai_cot.append('Id')
loai_cot.append('SalePrice')

test

In [17]:
phan_tram_thieu_test = data_test.isnull().sum() * 100 / len(data_test)
phan_tram_thieu_test = phan_tram_thieu_test[phan_tram_thieu_test > 0].sort_values(ascending=False)
print(phan_tram_thieu_test)

PoolQC          99.794380
MiscFeature     96.504455
Alley           92.666210
Fence           80.123372
MasVnrType      61.274846
FireplaceQu     50.034270
LotFrontage     15.558602
GarageCond       5.346127
GarageYrBlt      5.346127
GarageQual       5.346127
GarageFinish     5.346127
GarageType       5.209047
BsmtCond         3.084304
BsmtExposure     3.015764
BsmtQual         3.015764
BsmtFinType1     2.878684
BsmtFinType2     2.878684
MasVnrArea       1.028101
MSZoning         0.274160
BsmtFullBath     0.137080
BsmtHalfBath     0.137080
Functional       0.137080
Utilities        0.137080
GarageCars       0.068540
GarageArea       0.068540
TotalBsmtSF      0.068540
KitchenQual      0.068540
BsmtUnfSF        0.068540
BsmtFinSF2       0.068540
BsmtFinSF1       0.068540
Exterior2nd      0.068540
Exterior1st      0.068540
SaleType         0.068540
dtype: float64


In [18]:
loai_cot_ = phan_tram_thieu_test[phan_tram_thieu_test > 0].index.tolist()
phan_tram_thieu_test.drop(columns=loai_cot_, inplace=True) #This line excludes features_to_exclude list of columns from training data
loai_cot_.append('Id')
loai_cot_.append('SalePrice')

In [19]:
loai_cot=loai_cot+loai_cot_

In [20]:
loai_cot =list(set(loai_cot))

In [21]:
print(loai_cot)

['MSZoning', 'MasVnrArea', 'SaleType', 'BsmtCond', 'PoolQC', 'Exterior2nd', 'Electrical', 'BsmtFinSF2', 'BsmtFullBath', 'BsmtQual', 'BsmtFinType1', 'SalePrice', 'BsmtHalfBath', 'MasVnrType', 'BsmtUnfSF', 'GarageArea', 'Exterior1st', 'TotalBsmtSF', 'Id', 'BsmtFinType2', 'Alley', 'KitchenQual', 'BsmtFinSF1', 'Fence', 'GarageCars', 'MiscFeature', 'GarageYrBlt', 'BsmtExposure', 'Functional', 'GarageFinish', 'Utilities', 'FireplaceQu', 'GarageCond', 'LotFrontage', 'GarageQual', 'GarageType']


In [22]:
print()

In [23]:
cac_cot_kieu_ky_tu_train = [col for col in cac_cot_kieu_ky_tu_train if col not in loai_cot]
cac_cot_kieu_so_train = [col for col in cac_cot_kieu_so_train if col not in loai_cot]

In [24]:
print(cac_cot_kieu_ky_tu_train)

['Street', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'PavedDrive', 'SaleCondition']


In [25]:
print(cac_cot_kieu_so_train)

['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']


In [26]:
cac_cot_con_lai = cac_cot_kieu_ky_tu_train+cac_cot_kieu_so_train
cac_cot_con_lai.append("SalePrice")

In [27]:
data_train_new = data_train[cac_cot_con_lai]

In [28]:
data_train_new.info

<bound method DataFrame.info of      Street LotShape LandContour LotConfig LandSlope Neighborhood Condition1  \
0      Pave      Reg         Lvl    Inside       Gtl      CollgCr       Norm   
1      Pave      Reg         Lvl       FR2       Gtl      Veenker      Feedr   
2      Pave      IR1         Lvl    Inside       Gtl      CollgCr       Norm   
3      Pave      IR1         Lvl    Corner       Gtl      Crawfor       Norm   
4      Pave      IR1         Lvl       FR2       Gtl      NoRidge       Norm   
...     ...      ...         ...       ...       ...          ...        ...   
1455   Pave      Reg         Lvl    Inside       Gtl      Gilbert       Norm   
1456   Pave      Reg         Lvl    Inside       Gtl       NWAmes       Norm   
1457   Pave      Reg         Lvl    Inside       Gtl      Crawfor       Norm   
1458   Pave      Reg         Lvl    Inside       Gtl        NAmes       Norm   
1459   Pave      Reg         Lvl    Inside       Gtl      Edwards       Norm   

     Co

In [29]:
pourcentage(data_train_new)

,missing_number,types,duplicate,NAN
Street,0,object,0,0
LotShape,0,object,0,0
LandContour,0,object,0,0
LotConfig,0,object,0,0
LandSlope,0,object,0,0
Neighborhood,0,object,0,0
Condition1,0,object,0,0
Condition2,0,object,0,0
BldgType,0,object,0,0
HouseStyle,0,object,0,0


In [30]:
def ma_hoa(x):
    from sklearn.calibration import LabelEncoder
    label_encoder = LabelEncoder()
    # Áp dụng mã hóa nhãn cho cột 'Street_Type'
    data_train_new.loc[:,x] = label_encoder.fit_transform(data_train_new[x])


In [31]:
for cot in cac_cot_kieu_ky_tu_train:
    ma_hoa(cot)

In [32]:
data_train_new.columns

Index(['Street', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond', 'Foundation',
       'Heating', 'HeatingQC', 'CentralAir', 'PavedDrive', 'SaleCondition',
       'MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice'],
      dtype='object')

test

test

In [33]:
pourcentage(data_test)

,missing_number,types,duplicate,NAN
Id,0,int64,0,0
MSSubClass,0,int64,0,0
MSZoning,4,object,0,4
LotFrontage,227,float64,0,227
LotArea,0,int64,0,0
...,...,...,...,...
MiscVal,0,int64,0,0
MoSold,0,int64,0,0
YrSold,0,int64,0,0
SaleType,1,object,0,1


In [34]:
cac_cot_test = data_test.columns

In [35]:
cac_cot_con_lai_test = [col for col in cac_cot_test if col not in loai_cot]

In [36]:
data_test_new = data_test[cac_cot_con_lai_test]

In [37]:
cac_cot_kieu_ky_tu_test = xac_dinh_cac_cot_kieu_ky_tu(data_test_new)
cac_cot_kieu_so_test = xac_dinh_cac_cot_kieu_so(data_test_new)

In [38]:
def ma_hoa(x):
    from sklearn.calibration import LabelEncoder
    label_encoder = LabelEncoder()
    # Áp dụng mã hóa nhãn cho cột 'Street_Type'
    data_test_new.loc[:,x] = label_encoder.fit_transform(data_test_new[x])

In [39]:
for cot in cac_cot_kieu_ky_tu_test:
    ma_hoa(cot)

In [40]:
data_test_new.head

<bound method NDFrame.head of       MSSubClass  LotArea Street LotShape LandContour LotConfig LandSlope  \
0             20    11622      1        3           3         4         0   
1             20    14267      1        0           3         0         0   
2             60    13830      1        0           3         4         0   
3             60     9978      1        0           3         4         0   
4            120     5005      1        0           1         4         0   
...          ...      ...    ...      ...         ...       ...       ...   
1454         160     1936      1        3           3         4         0   
1455         160     1894      1        3           3         4         0   
1456          20    20000      1        3           3         4         0   
1457          85    10441      1        3           3         4         0   
1458          60     9627      1        3           3         4         1   

     Neighborhood Condition1 Condition2  ... 

In [41]:
data_train_new.head

<bound method NDFrame.head of      Street LotShape LandContour LotConfig LandSlope Neighborhood Condition1  \
0         1        3           3         4         0            5          2   
1         1        3           3         2         0           24          1   
2         1        0           3         4         0            5          2   
3         1        0           3         0         0            6          2   
4         1        0           3         2         0           15          2   
...     ...      ...         ...       ...       ...          ...        ...   
1455      1        3           3         4         0            8          2   
1456      1        3           3         4         0           14          2   
1457      1        3           3         4         0            6          2   
1458      1        3           3         4         0           12          2   
1459      1        3           3         4         0            7          2   

     Cond

In [42]:
dac_diem=['Street', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond', 'Foundation',
       'Heating', 'HeatingQC', 'CentralAir', 'PavedDrive', 'SaleCondition',
       'MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']
du_doan=['SalePrice']

In [43]:
X_train = data_train_new[dac_diem]
y_train = data_train_new[du_doan]

In [44]:
from sklearn import tree
model=tree.DecisionTreeClassifier(criterion = "entropy",
random_state = 100,
max_depth = 4,
min_samples_leaf = 10)

In [45]:
model = model.fit(X_train,y_train)

In [46]:
# tree.plot_tree(model)

In [47]:
# print(tree.export_text(model,show_weights = True))

In [48]:
X_test = data_test_new[dac_diem]

In [49]:
X_test.head(2)

,Street,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,...,Fireplaces,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,1,3,3,4,0,12,1,2,0,2,...,0,140,0,0,0,120,0,0,6,2010
1,1,0,3,0,0,12,2,2,0,2,...,0,393,36,0,0,0,0,12500,6,2010


In [50]:
y_pred = model.predict(X_test)
print("Giá trị mô hình dự đoán được:\n " ,y_pred)

Giá trị mô hình dự đoán được:
  [110000 140000 190000 ... 140000 110000 214000]


In [51]:
sample_submission_df = pd.read_csv('sample_submission.csv')
sample_submission_df['SalePrice'] = model.predict(X_test)
sample_submission_df.to_csv('submission.csv', index=False)
sample_submission_df.head()

,Id,SalePrice
0,1461,110000
1,1462,140000
2,1463,190000
3,1464,190000
4,1465,155000


In [53]:
# with open('house-prices-advanced-regression-techniques/decision_tree_model.pkl', 'wb') as f:
#     pickle.dump(model, f)

In [55]:
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression




# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Khởi tạo mô hình Logistic Regression
model2 = LogisticRegression()

# Huấn luyện mô hình trên tập huấn luyện
model2.fit(X_train, y_train)

# Dự đoán nhãn của tập kiểm tra
y_pred = model2.predict(X_test)



C:\Users\ketoan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ketoan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [59]:
with open('logic_model.pkl', 'wb') as m:
    pickle.dump(model2, m)